# Loading data

In [373]:
import pandas as pd
order_products__train = pd.read_csv("../instacart_2017_05_01/order_products__train.csv")
order_products__prior = pd.read_csv("../instacart_2017_05_01/order_products__prior.csv")
products = pd.read_csv("../instacart_2017_05_01/products.csv")
orders = pd.read_csv("../instacart_2017_05_01/orders.csv")

In [374]:
orders['eval_set'].unique()

array(['prior', 'train', 'test'], dtype=object)

# Train data

In [375]:
orders_train = orders[orders['eval_set'] == 'train'].reset_index()

In [376]:
orders_train.head()

,index,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,10,1187899,1,train,11,4,8,14.0
1,25,1492625,2,train,15,1,11,30.0
2,49,2196797,5,train,5,0,11,6.0
3,74,525192,7,train,21,2,11,6.0
4,78,880375,8,train,4,1,14,10.0


In [377]:
orders_train = orders_train[['user_id', 'order_id']]

In [378]:
orders_train['order_id'].unique().shape[0]

131209

In [379]:
order_products__train['order_id'].unique().shape[0]

131209

In [380]:
order_products__train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [381]:
order_products__train = order_products__train[['order_id', 'product_id', 'reordered']]

In [382]:
order_products__train = order_products__train.groupby('order_id')['product_id'].apply(list).reset_index()

In [383]:
order_products__train.head()

,order_id,product_id
0,1,"[49302, 11109, 10246, 49683, 43633, 13176, 472..."
1,36,"[39612, 19660, 49235, 43086, 46620, 34497, 486..."
2,38,"[11913, 18159, 4461, 21616, 23622, 32433, 2884..."
3,96,"[20574, 30391, 40706, 25610, 27966, 24489, 39275]"
4,98,"[8859, 19731, 43654, 13176, 4357, 37664, 34065..."


In [384]:
user_products__train = pd.merge(order_products__train, orders_train, on='order_id')

In [385]:
user_products__train.head()

,order_id,product_id,user_id
0,1,"[49302, 11109, 10246, 49683, 43633, 13176, 472...",112108
1,36,"[39612, 19660, 49235, 43086, 46620, 34497, 486...",79431
2,38,"[11913, 18159, 4461, 21616, 23622, 32433, 2884...",42756
3,96,"[20574, 30391, 40706, 25610, 27966, 24489, 39275]",17227
4,98,"[8859, 19731, 43654, 13176, 4357, 37664, 34065...",56463


In [386]:
user_products__train = user_products__train[['user_id', 'product_id']]

In [387]:
user_products__train.to_csv("user_products__train.csv", index_label=False)

In [388]:
user_products__train.shape

(131209, 2)

In [393]:
user_products__train = pd.read_csv("user_products__train.csv")
user_products__train.shape

(131209, 2)

In [395]:
user_products__train.dtypes

user_id        int64
product_id    object
dtype: object

In [396]:
user_products__train.set_index('user_id', inplace=True)

In [400]:
user_products__train

,product_id
user_id,
112108,"[49302, 11109, 10246, 49683, 43633, 13176, 472..."
79431,"[39612, 19660, 49235, 43086, 46620, 34497, 486..."
42756,"[11913, 18159, 4461, 21616, 23622, 32433, 2884..."
17227,"[20574, 30391, 40706, 25610, 27966, 24489, 39275]"
56463,"[8859, 19731, 43654, 13176, 4357, 37664, 34065..."
125030,"[27104, 21174, 41860, 38273, 47209, 5876, 2921..."
182389,"[18394, 37766, 13176, 6236, 5077, 8153, 43772,..."
98711,"[1194, 5578, 38159, 10305, 38557]"
51011,"[28199, 24852, 29883, 28427, 7754, 39947, 4730..."


In [428]:
user_products__train.iloc[0]

product_id    [49302, 11109, 10246, 49683, 43633, 13176, 472...
Name: 112108, dtype: object

# Prior data

In [28]:
orders_prior = orders[orders['eval_set'] == 'prior']
orders_prior['order_id'].unique().shape

(3214874,)

In [29]:
order_products__prior['order_id'].unique().shape[0]

3214874

In [35]:
user_products__prior = pd.merge(orders_prior, order_products__prior, on='order_id')
user_products__prior.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


In [37]:
user_products__prior = user_products__prior[['user_id', 'product_id']]
user_products__prior = user_products__prior.groupby(['user_id', 'product_id']).size().reset_index()
user_products__prior.head()

,user_id,product_id,0
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


In [39]:
user_products__prior = user_products__prior.rename(columns={0:'QTY'})

In [40]:
user_products__prior.to_csv('user_products__prior.csv', index_label=False)

In [168]:
user_products__prior = pd.read_csv('user_products__prior.csv')

# Sparse matrix

In [173]:
columns_to_cat = ['user_id', 'product_id']
for column in columns_to_cat:
    user_products__prior[column] = user_products__prior[column].astype('category')

In [113]:
from scipy.sparse import coo_matrix, save_npz
user_product_quantity = coo_matrix((user_products__prior["QTY"],
                                (user_products__prior["product_id"].cat.codes.copy(),
                                user_products__prior["user_id"].cat.codes.copy())))

# Here I'm using cat.codes to assign a coorinate sytem to users and product_id, then mappin the values onto a matrix.
# So the matrix values for (user, product_id) in cooridinate space will be the amound of product x user y bought
# Make a converter later to convert from user/product id numbers to matrix coordinates

save_npz('sparse.npz', user_product_quantity)

In [ ]:
from scipy.sparse import load_npz
user_product_quantity = load_npz('sparse.npz').tocsr().astype(np.float32)

In [114]:
user_product_quantity = user_product_quantity.tocsr().astype(np.float32)

Check later if you should demean data like [here](https://beckernick.github.io/matrix-factorization-recommender/)

# Model

In [556]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(user_product_quantity, k=50)
Vt = Vt.T * sigma

# Predictions

### Converters

In [218]:
user_products__prior['user_id'].cat.categories

Int64Index([     1,      2,      3,      4,      5,      6,      7,      8,
                 9,     10,
            ...
            206200, 206201, 206202, 206203, 206204, 206205, 206206, 206207,
            206208, 206209],
           dtype='int64', length=206209)

In [220]:
user_products__prior.head()

,user_id,product_id,QTY
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


### Converter dictionaries

In [286]:
# Put in id for product and user—get out coorinates for sparse matrix
user_to_coords = {user_id:i for i, user_id in enumerate(user_products__prior['user_id'].cat.categories)}
coords_to_product = {j:product_id for j, product_id in enumerate(user_products__prior['product_id'].cat.categories)} 

In [288]:
# Recommener function will output product_ids. Use this dict to then get human names for food
product_id_to_name_dict = pd.Series(products['product_name'].values, index=products['product_id']).to_dict()

### Recommender

In [548]:
def get_recommended_items(user_id, n, U, sigma, Vt):
    user_coord = user_to_coords[user_id]
    product_scores = Vt[user_coord].dot(U.T)
    #top_picks = sorted(recommended_items, reverse=True)[:n]
    #This is a more efficient way of doing it, don't really care about order—just want top n
    top_pick_coords = np.argpartition(product_scores, -n)[-n:]
    top_scores = product_scores[top_pick_coords]
    top_products = [coords_to_product[coord] for coord in top_pick_coords]
    
    return top_products

In [297]:
def product_id_to_name(product_ids):
    return [product_id_to_name_dict[id] for id in product_ids]
    

In [532]:
from ast import literal_eval
def get_real_items(user_id):
    return literal_eval(user_products__train.loc[user_id][0])
    

In [533]:
predicttest = product_id_to_name(get_recommended_items(21, 100))

In [534]:
realtest = product_id_to_name(get_real_items(21))

In [538]:
realtest

{'Dry Roasted 100 Calorie Packs Almonds',
 'Fridge Pack Cola',
 'Pepperidge Farm® Goldfish Xtra Cheddar Baked Snack Crackers',
 'Sparkling Water Grapefruit',
 'Sugarfree Energy Drink',
 'Zero Calories Berry Nutrient Enhanced Water'}

In [535]:
predicttest = set(predicttest)
realtest = set(realtest)
if len(realtest) == 0:
    print('oy')
len(predicttest.intersection(realtest)) / len(realtest)

0.3333333333333333

# Evaluator

### Test subjects

In [526]:
test_subjects = user_products__train.index

In [545]:
def get_model_accuracy(user_ids, n, U, sigma, Vt):
    accuracies = []
    for user_id in user_ids:
        recommended_items = set(get_recommended_items(user_id, n, U, sigma, Vt))
        real_items = set(get_real_items(user_id))
        accuracy = len(recommended_items.intersection(real_items)) / len(real_items)
        accuracies.append(accuracy)
        model_accuracy = np.mean(accuracies)
    return model_accuracy

In [549]:
get_model_accuracy(test_subjects, 10, U, sigma, Vt)

0.1425822929799591

In [542]:
get_model_accuracy(test_subjects)

0.1425822929799591

for k = 50, model accuracy is 0.14

In [537]:
model_accuracy

0.1425822929799591

## Hyperparameter optimization

In [554]:
k_values = [1, 25, 50 ,100, 150, 200]
for k in k_values:
    U, sigma, Vt = svds(user_product_quantity, k)
    Vt = Vt.T * sigma
    
    model_accuracy = get_model_accuracy(test_subjects, 10, U, sigma, Vt)
    
    print(f"{k}: {model_accuracy}")

50: 0.1425814320408241
100: 0.16813466095533894
150: 0.1828590552210617
200: 0.19352852970690326


Honestly not sure why it skipped 1 and 25, thought k just needed to be bigger than 1

50: 0.1425814320408241

100: 0.16813466095533894

150: 0.1828590552210617

200: 0.19352852970690326

In [555]:
k_values = [250, 300, 350, 400, 450]
for k in k_values:
    U, sigma, Vt = svds(user_product_quantity, k)
    Vt = Vt.T * sigma
    
    model_accuracy = get_model_accuracy(test_subjects, 10, U, sigma, Vt)
    
    print(f"{k}: {model_accuracy}")

250: 0.20264273917007064
300: 0.20984979648356833
350: 0.2163621196679199
400: 0.2215989923078134
450: 0.2262816610793348


Starting to taper off 
 
250: 0.20264273917007064

300: 0.20984979648356833

350: 0.2163621196679199

400: 0.2215989923078134

450: 0.2262816610793348